In this notebook we're going to add the target variable. The target variable is defined by ourselfes - based on:
- research
- data analysis
- clustering

We detected 2 main infringements:
- hosts without license offering short term stays (under 32 days)
- hosts who claim they're private but have more than 3 listings 

We are now going to add a column (suspicious_or_not):
- 0: none of the mentioned infringements detected
- 1: either one or even both of the mentioned infringements detected

At the end of this notebook we're going to clean the dataset from all calculated fields (either calculated by ourselves of by insideairbnb!) except for our target variable. 

The idea is that our model should (hopefully) classify correctly based only on information that every normal user can easily get from the listing's airbnb website, without background knowledge or calculations!

In [1]:
# import libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# load the data
bcn_original = pd.read_csv('../data/listings_bcn.csv')
scraped_host_type = pd.read_csv('../data/airbnb2.csv')

In [3]:
# join both datasets
scraped_host_type.rename(columns = {'url':'listing_url'}, inplace = True)
bcn=bcn_original.merge(scraped_host_type, on='listing_url', how='outer')
bcn = bcn.drop(['Unnamed: 0'], axis=1)

In [4]:
# remove all rows with NaN values in host_type column
bcn = bcn.dropna(subset=['host_type'])

In [5]:
# remove all rows with not_found values in host_type column
bcn = bcn[bcn['host_type']!='not_found']

In [6]:
# impute missing license with "no license"
bcn['license'] = bcn['license'].fillna('no license')

In [7]:
def is_valid_license(license):
    if 'HUTB' in license:
        return 'ok'
    if 'HB' in license:
        return 'ok'
    if 'AJ' in license:
        return 'ok'
    elif 'exempt' in license.lower():
        return 'claims exempt'
    else:
        return 'no license'

In [8]:
# add status_license column
bcn['status_license'] = bcn.apply(lambda row: is_valid_license(row['license']), axis=1)

In [9]:
# add own calculation of total listings
bcn['total_listings'] = bcn.groupby('host_id')['id'].transform('size')

In [10]:
def license_infringement(row):
    if row['status_license'] == 'ok':
        return 0
    if row['status_license'] == 'no license' and row['minimum_nights'] >31:
        return 0
    if row['status_license'] == 'claims exempt' and row['minimum_nights'] >31:
        return 0
    else:
        return 1

In [11]:
#add column license infringement detected or not
bcn['license_infringement'] = bcn.apply(lambda row: license_infringement(row), axis=1)

In [12]:
def claim_private_infringement(row):
    if row['host_type'] == 'private' and row['total_listings'] >3:
        return 1
    else:
        return 0

In [13]:
#add column claim private infringement detected or not
bcn['claim_private_infringement'] = bcn.apply(lambda row: claim_private_infringement(row), axis=1)

In [14]:
def suspicious_or_not(row):
    if row['license_infringement'] == 0 and row['claim_private_infringement'] == 0:
        return 0
    else:
        return 1

In [15]:
# add target column
bcn['suspicious_or_not'] = bcn.apply(lambda row: suspicious_or_not(row), axis=1)

In [16]:
#double-check
bcn[['host_type', 'total_listings', 'claim_private_infringement', 'status_license', 'minimum_nights', 'license_infringement', 'suspicious_or_not']].head(30)

,host_type,total_listings,claim_private_infringement,status_license,minimum_nights,license_infringement,suspicious_or_not
0,private,1,0,no license,1,1,1
1,private,1,0,claims exempt,2,1,1
2,private,4,1,ok,4,0,1
3,commercial,6,0,ok,1,0,0
4,private,1,0,claims exempt,2,1,1
5,private,8,1,ok,1,0,1
6,private,2,0,ok,3,0,0
7,commercial,27,0,no license,32,0,0
8,commercial,84,0,ok,3,0,0
9,private,1,0,no license,2,1,1


In [19]:
# check for distribution of target variable
bcn['suspicious_or_not'].value_counts(normalize=True)

suspicious_or_not
0    0.543298
1    0.456702
Name: proportion, dtype: float64

#### To do:
- train test split
- preprocessing
- baseline model

In [20]:
bcn.columns

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'source', 'name',
       'description', 'neighborhood_overview', 'picture_url', 'host_id',
       'host_url', 'host_name', 'host_since', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_thumbnail_url', 'host_picture_url',
       'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'neighbourhood',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms',
       'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
       'minimum_nights', 'maximum_nights', 'minimum_minimum_nights',
       'maximum_minimum_nights', 'minimum_maximum_nights',
       'maximum_maximum_nights', 'minimum_nights_avg_ntm',
       'maximum_nights_avg_ntm', 'ca

In [22]:
# delete all calculated fields to proceed with a clean dataset

bcn_ready_for_model = bcn.drop(['listing_url', 'scrape_id', 'last_scraped', 'source', 'host_url', 'minimum_minimum_nights',
       'maximum_minimum_nights', 'minimum_maximum_nights',
       'maximum_maximum_nights', 'minimum_nights_avg_ntm',
       'maximum_nights_avg_ntm', 'availability_30', 'availability_60', 'availability_90',
       'availability_365', 'calendar_last_scraped', 'number_of_reviews_ltm', 'number_of_reviews_l30d', 'first_review',
       'last_review', 'calculated_host_listings_count',
       'calculated_host_listings_count_entire_homes',
       'calculated_host_listings_count_private_rooms',
       'calculated_host_listings_count_shared_rooms', 'reviews_per_month', 'status_license', 'total_listings', 'license_infringement',
       'claim_private_infringement'], axis=1)

In [25]:
bcn_ready_for_model.columns

Index(['id', 'name', 'description', 'neighborhood_overview', 'picture_url',
       'host_id', 'host_name', 'host_since', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_thumbnail_url', 'host_picture_url',
       'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'neighbourhood',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms',
       'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
       'minimum_nights', 'maximum_nights', 'calendar_updated',
       'has_availability', 'number_of_reviews', 'review_scores_rating',
       'review_scores_accuracy', 'review_scores_cleanliness',
       'review_scores_checkin', 'review_scores_communication',
       'review_scores_location', 're

In [26]:
bcn_ready_for_model.to_csv('df_modelling.csv') 